# Proyecto 1
* Flavio Galán 22386
* Josue Say
* Isabella Miralles

## Descarga de los Datos
Para descargar los datos se utiliza Selenium y Python, además se tiene un sistema de cacheo que funciona en base al archivo `links_cache.txt`. Si este archivo ya existe entonces no se realizará ninguna descarga de los datos. Los datos se guardan en varios archivos txt. Cada uno representa la columna del dataframe y todos los datos de esa columna. No se tienen en un formato JSON ni CSV sino que se guardan directamente en formato python para la facilidad de extracción usando el mismo lenguaje. Ya que estos son los datos sucios no hay ningún problema con guardarlos así, los datos limpios ya se guardarán en un formaton más estandarizado.

In [ ]:
import os
import time
import subprocess
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

cache_file_name = "links_cache.txt"
zip_dir = "zips/"
files_dir = "data/"

codigo = []
distrito = []
departamento = []
municipio = []
establecimiento = []
direccion = []
telefono = []
supervisor = []
director = []
nivel = []
sector = []
area = []
status = []
modalidad = []
jornada = []
plan = []
departamental = []
agggrArrays = [
    codigo,
    distrito,
    departamento,
    municipio,
    establecimiento,
    direccion,
    telefono,
    supervisor,
    director,
    nivel,
    sector,
    area,
    status,
    modalidad,
    jornada,
    plan,
    departamental,
]
filenames = [
    "codigo",
    "distrito",
    "departamento",
    "municipio",
    "establecimiento",
    "direccion",
    "telefono",
    "supervisor",
    "director",
    "nivel",
    "sector",
    "area",
    "status",
    "modalidad",
    "jornada",
    "plan",
    "departamental",
]

if os.path.exists(cache_file_name):
    for idx in range(len(filenames)):
        filename = filenames[idx]
        data = agggrArrays[idx]
        with open(zip_dir + filename + ".txt", "r") as file:
            content = file.read()
            data = eval(content)
else:
    with webdriver.Firefox() as driver:
        driver.implicitly_wait(3)

        driver.get("http://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/")
        assert "Búsqueda de centros" in driver.title

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']")
            )
        )
        deptSelect = driver.find_element(
            By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']"
        )
        deptSelect = Select(deptSelect)
        depts = deptSelect.options
        print("Getting depts")
        for idx in range(len(depts) - 1):
            print("Selecting dept", idx + 1)
            deptSelect = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']"
            )
            deptSelect = Select(deptSelect)
            deptSelect.select_by_index(idx + 1)
            print("Finding education level")
            levelSelect = driver.find_element(
                By.XPATH, '//*[@id="_ctl0_ContentPlaceHolder1_cmbNivel"]'
            )
            levelSelect = Select(levelSelect)
            print("Selecting diversificado")
            levelSelect.select_by_value("46")  # 46 es Diversificado
            print("Finding search button")
            btn = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_IbtnConsultar']"
            )
            btn.click()
            print("Clicking button")

            # Wait for results
            print("Esperando por resultados")
            time.sleep(5)
            print("Asumimos que se obtuvieron los resultados")

            table = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_dgResultado']"
            )
            rows = table.find_elements(By.XPATH, ".//tr")
            for rowIdx in range(
                1, len(rows) - 1
            ):  # La última fila siempre es una vacía
                cells = rows[rowIdx].find_elements(By.XPATH, ".//td")

                for cellIdx in range(1, len(cells)):
                    agggrArrays[cellIdx - 1].append(
                        cells[cellIdx].get_attribute("textContent")
                    )

            # print("Obtained following names")
            # print(establecimiento)
            # exit(1)

        print("Saving cache...")
        subprocess.run(["mkdir", "-p", zip_dir])
        for idx in range(len(filenames)):
            filename = filenames[idx]
            data = agggrArrays[idx]

            lines = ["["]
            for val in data:
                line = f"\t'''{val}''',\n"
                lines.append(line)
            lines.append("]")

            with open(zip_dir + filename + ".txt", "w") as file:
                file.writelines(lines)

        with open(cache_file_name, "w") as file:
            file.write("DELETE ME IF YOU WANT TO REDOWNLOAD DATA!")



Si ya se tienen los datos descargados pero se necesita generar el dataframe entonces corre el siguiente script:

In [ ]:
df = pd.DataFrame(
    {colName: colData for (colName, colData) in zip(filenames, agggrArrays)}
)
print("The resulting Dataframe is")
print(df)

## Descripción de los Datos
Se describen los datos y las transformaciones necesarias que se les realizaran a continuación.

## Limpieza de los datos
Se procede a ejecutar las transformaciones previamente ideadas y a unificar todos los datasets en uno solo.